# Preprocessing for Ames Housing Dataset

## Prepare the Ames Housing Data for AutoML and final modeling.

## 1. Transforming the target variable (SalePrice)


### Prepared by Ian Kidwell

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
csv_path = "../data/AmesHousing.csv"

# Load CSV
df = pd.read_csv(csv_path)

# Quick check
print(df.head())

In [0]:
# Log-transform the target variable
df['SalePrice_log'] = np.log1p(df['SalePrice'])  # log1p handles zero values safely

# Quick visualization to see the effect
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
sns.histplot(df['SalePrice'], kde=True, bins=50)
plt.title('Original SalePrice Distribution')

plt.subplot(1,2,2)
sns.histplot(df['SalePrice_log'], kde=True, bins=50, color='orange')
plt.title('Log-Transformed SalePrice Distribution')

plt.tight_layout()
plt.show()

## 2. Handling Missing Values

In [0]:
# Identify numeric and categorical columns
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Optional: remove target variable from features
if 'SalePrice' in numeric_features:
    numeric_features.remove('SalePrice')

# Quick check
print("Numeric features:", numeric_features[:10], "...")
print("Categorical features:", categorical_features[:10], "...")


In [0]:
# For numeric features
for col in numeric_features:
    df[col] = df[col].fillna(df[col].median())

# For categorical features
for col in categorical_features:
    df[col] = df[col].fillna("Missing")


# Quick check to confirm
missing_summary = df.isnull().sum()[df.isnull().sum() > 0]
print("Remaining missing values:\n", missing_summary)


## 3. Categorical Feature Encoding

In [0]:
# List all categorical columns
for col in categorical_features:
    print(f"{col}: {df[col].unique()}\n")


In [0]:
from sklearn.preprocessing import OrdinalEncoder

# Define ordinal mappings
qual_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'Missing': 0}
bsmt_exposure_mapping = {'No': 0, 'Mn': 1, 'Av': 2, 'Gd': 3, 'Missing': 0}
bsmt_fin_mapping = {'Unf': 0, 'LwQ': 1, 'Rec': 2, 'BLQ': 3, 'ALQ': 4, 'GLQ': 5, 'Missing': 0}
garage_finish_mapping = {'Missing': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}
paved_drive_mapping = {'N': 0, 'P': 1, 'Y': 2}
functional_mapping = {'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8}

# Apply ordinal encoding
ordinal_features = ['Exter Qual', 'Exter Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
                    'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating QC', 'Kitchen Qual',
                    'Fireplace Qu', 'Garage Finish', 'Garage Qual', 'Garage Cond',
                    'Pool QC', 'Functional', 'Paved Drive']

df['Exter Qual'] = df['Exter Qual'].map(qual_mapping)
df['Exter Cond'] = df['Exter Cond'].map(qual_mapping)
df['Bsmt Qual'] = df['Bsmt Qual'].map(qual_mapping)
df['Bsmt Cond'] = df['Bsmt Cond'].map(qual_mapping)
df['Bsmt Exposure'] = df['Bsmt Exposure'].map(bsmt_exposure_mapping)
df['BsmtFin Type 1'] = df['BsmtFin Type 1'].map(bsmt_fin_mapping)
df['BsmtFin Type 2'] = df['BsmtFin Type 2'].map(bsmt_fin_mapping)
df['Heating QC'] = df['Heating QC'].map(qual_mapping)
df['Kitchen Qual'] = df['Kitchen Qual'].map(qual_mapping)
df['Fireplace Qu'] = df['Fireplace Qu'].map(qual_mapping)
df['Garage Finish'] = df['Garage Finish'].map(garage_finish_mapping)
df['Garage Qual'] = df['Garage Qual'].map(qual_mapping)
df['Garage Cond'] = df['Garage Cond'].map(qual_mapping)
df['Pool QC'] = df['Pool QC'].map(qual_mapping)
df['Functional'] = df['Functional'].map(functional_mapping)
df['Paved Drive'] = df['Paved Drive'].map(paved_drive_mapping)

# Remaining categorical features → one-hot encode
nominal_features = [col for col in categorical_features if col not in ordinal_features]
df = pd.get_dummies(df, columns=nominal_features, drop_first=True)

# Quick check
print(df.head())


## 4. Scaling Numeric Features (for AutoML)

In [0]:
# 1. Create an unscaled copy
df_unscaled = df.copy()

from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Fit and transform numeric features
df[numeric_features] = scaler.fit_transform(df[numeric_features])

# Quick check
df[numeric_features].head()


## 5. Feature Selection

In [0]:
# List all features in the dataset
all_features = df.columns.tolist()
print("Total features:", len(all_features))
print(all_features)


In [0]:
import os

# Drop only non-informative columns (Keeping your logic)
#df.drop(columns=['Order', 'PID'], inplace=True)
#df_unscaled.drop(columns=['Order', 'PID'], inplace=True)

# --- SAVE TO DATA FOLDER ---

# We add '../data/' to the filename to send it to the right folder
path_scaled = "../data/ames_preprocessed.csv"
path_unscaled = "../data/ames_preprocessed_unscaled.csv"

# Save preprocessed dataset for AutoML
df.to_csv(path_scaled, index=False)
df_unscaled.to_csv(path_unscaled, index=False)

# Verification: Print exactly where they were saved
print(f"Saved file to: {os.path.abspath(path_scaled)}")
print(f"Saved file to: {os.path.abspath(path_unscaled)}")

In [0]:
import os

# Drop all text/object columns (Logic stays the same)
df_numeric_only = df.select_dtypes(include=['int64', 'float64']).copy()
df_numeric_only_unscaled = df_unscaled.select_dtypes(include=['int64', 'float64']).copy()

# Quick check
print(df_numeric_only.info())

# --- FIX: Save to the data folder ---
# We define the paths relative to this notebook (go up one level, then into data)
path_numeric = "../data/ames_preprocessed_numeric.csv"
path_numeric_unscaled = "../data/ames_preprocessed_numeric_unscaled.csv"

# Save the cleaned DataFrames
df_numeric_only.to_csv(path_numeric, index=False)
df_numeric_only_unscaled.to_csv(path_numeric_unscaled, index=False)

# Verification: Print the full path so you can confirm they landed in the right spot
print(f"Saved numeric-only CSV to: {os.path.abspath(path_numeric)}")
print(f"Saved unscaled CSV to:     {os.path.abspath(path_numeric_unscaled)}")

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# --- INPUT: Read from ../data/ ---
# This matches the file we saved in the previous cell
input_path = "../data/ames_preprocessed_numeric.csv"
df = pd.read_csv(input_path)

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# --- OUTPUT: Save to ../data/ ---
train_path = "../data/train.csv"
test_path = "../data/test.csv"

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

# Verification
print(f"Read data from: {os.path.abspath(input_path)}")
print(f"Saved Train to: {os.path.abspath(train_path)}")
print(f"Saved Test to:  {os.path.abspath(test_path)}")